In [ ]:
wd<-"/home/jovyan/zxli_SCI/result/WGCNA/WT.SCT.res02.domain_220311/topregulon_220411/"
if(!dir.exists(wd))
    dir.create(wd)
setwd(wd)
getwd()

In [ ]:
library(KernSmooth)
library(RColorBrewer)
library(plotly)
library(BiocParallel)
library(grid)
#library(ComplexHeatmap)
library(data.table)
library(dplyr)
library(tidyverse)
library(pheatmap)
library(scales)

In [4]:
save_pheatmap_pdf <- function(x, filename, width=10, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}

In [ ]:
### create module gene

In [ ]:
modgene<-read.csv("../thr100.sp18.hclust_average.min30.deep2.h015.34modules.full.gene.list.csv")#("min30.ch09.mh01/min30.cuth09.mergh01.modules.full.gene.list.csv")#("WT.s4.modules.mergheight045.full.gene.list.csv")#read.csv("WT.s4.hvg1w.modules.full.gene.list.csv")
rownames(modgene)<-modgene$gene
dim(modgene)
modgene<-modgene[!modgene$module=="grey",]
#modgene<-modgene[,-which(colnames(modgene)%in%c("X","gene"))]
head(modgene)
dim(modgene)
table(modgene$module)
length(unique(modgene$module))

In [ ]:
auc<-read.csv("/home/jovyan/result/pyscenic/WT.merge.replace_v2/SCT/auc_mtx.csv")
auc[1:5,1:5]
dim(auc)

In [ ]:
meta<-read.csv("/home/jovyan/result/Seurat/reg.CC/WT_replace_v2/res02_220310/WT.SCT.pc20.k50.res02.meta.data.csv")
rownames(meta)<-meta[,1]
head(meta)

In [10]:
meta<-cbind(meta,auc)

In [11]:
write.csv(meta,"WT.SCT.reg.pc20.k50.res02.auc.meta.csv")

In [ ]:
meta<-read.csv("WT.SCT.reg.pc20.k50.res02.auc.meta.csv")
rownames(meta)<-meta[,1]

In [ ]:
mods<-split(modgene$gene,modgene$module)
head(mods)

In [ ]:
regulon<-read.csv("/home/jovyan/result/pyscenic/WT.merge.replace_v2/SCT/regulons.csv")
head(regulon)

In [ ]:
rownames(regulon)<-regulon$tfs
head(regulon)
length(unique(regulon$motif))
dim(regulon)
length(unique(regulon$tfs))
regulon$targets<-as.character(regulon$targets)

In [ ]:
# get regulon genes list
regulon_list<-list()
for(i in regulon$tfs){
    regulon_list[[i]]<-c(i,unlist(strsplit(regulon[regulon$tfs==i,"targets"],",")))
}
length(regulon_list)

In [18]:
saveRDS(regulon_list,"WT.regulon.list.rds")

In [3]:
regulon_list<-readRDS("WT.regulon.list.rds")

In [ ]:
regulon_list[[1]]
head(regulon_list)

In [ ]:
r_genes<-unique(unlist(regulon_list))
length(r_genes)

In [ ]:
### 220411:change background genes from all module genes to SCT genes
##background genes
backgenes<-readRDS("/home/jovyan/result/cellchat/time_domain_220325/WT.SCT.data.rds")
backgenes<-rownames(backgenes)
head(backgenes)

In [ ]:
total_genes<-backgenes#unique(unlist(regulon_list))
t_l<-length(total_genes)
t_l

In [ ]:
### check if all module genes are contained in background genes
intgenes<-intersect(r_genes,total_genes)
length(intgenes)
table(r_genes %in% total_genes)

In [1]:
### create hypergeometric dataframe for each module
mod_regulon_df_list<-list()
#inter_mod_reg_df<-data.frame(row.names = names(mods),module=names(mods))
#c<-2
f<-"intersect_genes/"
if(!dir.exists(f))
    dir.create(f)

for(i in names(regulon_list)){
    inter_list<-list()
    #inter_mod_reg_df[,i]<-NA
    r_genes<-regulon_list[[i]]
    #r_genes<-intersect(r_genes,intgenes) ### filter regulon genes that not exist in all module gene background
    k<-length(r_genes)
    # find the module success gene number in target regulon,regulon number, remaining regulons size and module size
    hyper_df <- data.frame(matrix(nrow=length(mods),ncol = 9),row.names = names(mods))
    

    colnames(hyper_df) <- c("q",  #regulon in module
                            "m",  #module size
                            "n",  #regulon in background 
                            "k",  #regulon size
                            "p.val",
                            "p.val.ad",
                            "sig",
                            "ratio",
                            "score")

    for(j in names(mods)){
        m_genes<-mods[[j]]
        suc_genes<-intersect(m_genes,r_genes)
        suc_genes_l<-length(suc_genes)
        m<-length(m_genes)
        n<-t_l-m
        p<-round(phyper(suc_genes_l,m,n,k,lower.tail = F,log.p = F),4)
        r<-round(suc_genes_l/k,4)
        
        if(suc_genes_l==0){
            inter_list[[j]]<-""
        }else{
            inter_list[[j]]<-c(suc_genes)
        }
        
        
        hyper_df[j,c("q","m","n","k","p.val")]<-c(suc_genes_l,m,n,k,p)
        hyper_df$ratio<-round(hyper_df$q/hyper_df$k,4) #calculate the regulon enriched ratio in each module
        #hyper_df<-subset(hyper_df,ratio>0.2) #subset regulons that have 20% enrichment ratio in target modules
    }
    
    saveRDS(inter_list,paste0(f,i,".intersect_genes.in.modules.rds"))
    #inter_mod_reg_df<-as.data.frame(cbind(inter_mod_reg_df,inter_list))
    #colnames(inter_mod_reg_df)[c]<-i
    
    hyper_df$p.val.ad<-round(p.adjust(hyper_df$p.val,"bonferroni"),4)
    hyper_df$sig<-round((1-hyper_df$p.val.ad)/0.05,4)
    hyper_df[hyper_df$p.val.ad>=0.01,"sig"]<-0
    hyper_df$score<-hyper_df$sig*hyper_df$ratio
    mod_regulon_df_list[[i]]<-hyper_df
    #mod_regulon_df_list[[i]]$p.val.ad<-
    #mod_regulon_df_list[[i]]$sig<-round(mod_regulon_df_list[[i]])
    #dim(df)
    #head(df)
    #c<-c+1
}


In [ ]:
### create module-regulon p.val matrix
p_ma<-do.call(cbind.data.frame,mod_regulon_df_list)
head(p_ma)
p_ma<-as.matrix(p_ma[,grepl(".sig",colnames(p_ma))])
colnames(p_ma)<-gsub(".sig","",colnames(p_ma))
p_ma<-t(p_ma)
head(p_ma)

In [ ]:
# create ratio matrix
r_ma<-do.call(cbind.data.frame,mod_regulon_df_list)
head(r_ma)
r_ma<-as.matrix(r_ma[,grepl("ratio",colnames(r_ma))])
colnames(r_ma)<-gsub(".ratio","",colnames(r_ma))
r_ma<-t(r_ma)
head(r_ma)

In [ ]:
s_ma<-do.call(cbind.data.frame,mod_regulon_df_list)
head(s_ma)
s_ma<-as.matrix(s_ma[,grepl("score",colnames(s_ma))])
colnames(s_ma)<-gsub(".score","",colnames(s_ma))
s_ma<-t(s_ma)
head(s_ma)

In [116]:
write.csv(p_ma,"module_regulon.hyper.pad001.pval.ad_significance.matrix.csv")
write.csv(s_ma,"module-regulon.hyper.pad001.pval-ratio.score.matrix.csv")

In [ ]:
s_ma<-read.csv("module-regulon.hyper.pad001.pval-ratio.score.matrix.csv")
rownames(s_ma)<-s_ma[,1]
s_ma<-s_ma[,-1]
head(s_ma)

In [117]:
saveRDS(mod_regulon_df_list,"module.regulon.hypergeomic.test.list.pad001.rds")

In [5]:
mod_regulon_df_list<-readRDS("module.regulon.hypergeomic.test.list.rds")

In [ ]:
df<-do.call(rbind.data.frame,mod_regulon_df_list)
head(df)
df$module<-str_split(rownames(df),"\\.",simplify = T)[,2]
df$regulon<-str_split(rownames(df),"\\.",simplify = T)[,1]
##df<-df[!df$module=="",]
head(df)
dim(df)

In [119]:
#remove regulon size <=10
df<-subset(df,k>10)

In [120]:
top3<- df %>% subset(.,score>0) %>% group_by(module)  %>% top_n(.,wt = sig,n = 10) %>% top_n(.,wt = score,n = 3) #
    

In [ ]:
#top3<-ubset(top3,module %in% c("black","blue","brown","green","greenyellow",
#                               "lightcyan","magenta","pink","purple","red","yellow"))
top3<-arrange(top3,module)
head(top3)

In [ ]:
df3<-p_ma[unique(top3$regulon),unique(top3$module)]
#df2<-df2[,c("black","blue","brown","green","greenyellow","lightcyan","magenta","pink","purple","red","yellow")]
df3

In [124]:
col_anno<-data.frame(row.names = colnames(df3),"module"=colnames(df3))
module_col<-colnames(df3)
names(module_col)<-colnames(df3)
col_list<-list(module=module_col)

In [ ]:
options(repr.plot.width=7, repr.plot.height=12) # To set the figure size in Jupyter
#png("selected.module.pad001.selfdefine.top3.score.regulons.heatmap.png",width=7,height=12,res = 300)
p<-pheatmap(df3, name="Score",cluster_rows = F,cluster_cols = F,border_color = NA,#scale = "row",
         col=viridisLite::viridis(n = 50),annotation_col=col_anno,annotation_colors=col_list
         #colorRampPalette(c("darkred","white","skyblue"),space="rgb")(100),
                       #row_names_gp=grid::gpar(fontsize=11),cluster_columns=F,cluster_rows=F
        )
#dev.off()

In [ ]:
save_pheatmap_pdf(p,filename = "selected.module.pad001.selfdefine.top3.significance.regulons.heatmap.unscaled.pdf",width = 7,height = 12)

#### plot module top3 regulon dotplot on time_domain groups

In [ ]:
meta$time_domain<-paste0(meta$time,"_",meta$domain)
colnames(meta)
unique(meta$time_domain)

In [ ]:
meta$time_domain<-factor(meta$time_domain,levels = c(unique(meta$time_domain)))
levels(meta$time_domain)

In [149]:
meta %>% group_by(time_domain) %>% summarise_at(vars(colnames(meta)[49:ncol(meta)-1]),mean) ->td_mean

In [ ]:
td_mean$time<-str_split(td_mean$time_domain,"_",simplify = T)[,2]
td_mean$domain<-str_split(td_mean$time_domain,"_",simplify = T)[,3]
head(td_mean)

In [151]:
td_mean$time<-factor(td_mean$time,levels = c("72h","24h","3h","sham"))

In [ ]:
#colnames(td_mean)<-gsub("...$","",colnames(td_mean))
colnames(td_mean)<-gsub("\\.","-",colnames(td_mean),fixed = T)
head(td_mean)

In [153]:
write.csv(td_mean,"regulon.average.auc.score.time_domain.csv")

In [154]:
options(repr.plot.width=4.5,repr.plot.height=4)
library(ggplot2)
f2<-"module_top_regulon_dotplot_0412"
if(!dir.exists(f2))
    dir.create(f2)
for(i in paste0(top3$regulon,"...")){
    
    if(!i %in% colnames(td_mean))
        i<-gsub("-",".",i)
    ggplot(td_mean,aes(domain,time))+geom_point(aes_string(color=i),size=20)+
    scale_color_gradientn(colours = c("blue","white","orangered"))+theme_bw()+
    theme(axis.text = element_text(size = 12),panel.background = element_blank(),panel.grid = element_blank())
    ggsave(paste0(f2,"/",i,"_time_domain.mean.dotplot.png"),width = 4.5,height = 4,dpi = 300)
}

### plot module top3 regulon spatial plot

In [ ]:
#adjust spatial position between multiple samples
grep("x.ad2",colnames(meta))
meta$x.ad2<-NA
meta$y.ad2<-NA
for(i in 1:length(unique(meta$orig.ident))){
    m=(i-1)%/%4
    x.add=(i-4*m-1)*100
    y.add=150*(4-m)
    meta[meta$orig.ident==unique(meta$orig.ident)[i],"x.ad2"]<-meta[meta$orig.ident==unique(meta$orig.ident)[i],"x.axis"]+x.add
    meta[meta$orig.ident==unique(meta$orig.ident)[i],"y.ad2"]<-meta[meta$orig.ident==unique(meta$orig.ident)[i],"y.axis"]+y.add
}
range(meta$x.ad2)
range(meta$y.ad2)

In [ ]:
#dir.create("moduletop3.regulon_spatialplot")
options(repr.plot.width=20,repr.plot.height=16)
for(i in unique(top3$regulon)){
    ggplot(meta,aes_string("x.ad2","y.ad2",color=paste0(i,"...")))+geom_point(size=1)+
        #scale_colour_gradientn(colours=colorRampPalette(rev(brewer.pal(n = 11, name = "Spectral")))(50))+
        ggplot2::scale_colour_gradientn(colours = c('#0000ff','#3232ff','#6666ff','#9898ff','#ccccff',
                                              '#fffefe','#ffcccc','#ff9898','#ff6666','#ff3232','#ff0000'))+
        xlab(paste0("")) +
        ylab(paste0("")) + 
        theme(panel.background = element_blank(),
              panel.grid.major =element_blank(), 
              panel.grid.minor = element_blank(),
              axis.text.x = element_blank(),
              axis.text.y = element_blank()
          # axis.ticks.x = element_blank(),
    )#+
    #ggtitle(label = top3)
    #p<-SpatialFeaturePlot(mergda,features = c(i),min.cutoff=mean(mergda@meta.data[,i]),combine = F)
    #do.call(ggpubr::ggarrange,c(p,list(ncol=4,nrow=8,common.legend=T,legend="right")))
    ggsave(paste0("moduletop3.regulon_spatialplot/regulon_",i,".auc.score.spatial.png"),width=20,height=16,dpi=300)
    }